# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [25]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [26]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [27]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [28]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [29]:
# Create a Keyspace 

session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)

#### Set Keyspace

In [30]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.
1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4¶
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
Query

In [31]:
''' Create user_songs table based on the following requirements:
Get the the artist, song title and song's length in the music app history that was heard during \
sessionId = 338, and itemInSession = 4

Columns of the table - sessionId ,itemInSession ,artist , song_title ,song_length 
PRIMARY KEY consist of partition key sessionId, and clustering key itemInSession.  
Because of this, we could filter the data based on these attributes

'''

session.execute('''
                CREATE TABLE IF NOT EXISTS session_songs
                (sessionId int,itemInSession int,artist text, song_title text,song_length float,
                PRIMARY KEY(sessionId,itemInSession))
                ''')


In [32]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_songs (sessionId,itemInSession,artist,song_title,song_length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        artist,first_name,gender,itemInSession,last_name,length,level,location,sessionId,song,userId=line
        
        session.execute(query, (int(sessionId),int(itemInSession),artist,song,float(length)))
        

#### Do a SELECT to verify that the data have been inserted into each table

In [33]:
## SELECT statement to verify the data was entered into the table

results = session.execute("""SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4""")

for res in results:
    print(res.artist, res.song_title, res.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [34]:
'''
create user_songs table based on the following requirements:
Get the name of artist, song (sorted by itemInSession) and user (first and last name)\
for userid = 10, sessionid = 182

Columns of the table - userid,sessionId , artist, itemInSession ,artist , song_title ,song_length,first_name,last_name
PRIMARY KEY consist of composite partition key userid, sessionId, and clustering key itemInSession.  
We use two partition keys because multiple session ids  can be for one userId.

'''


session.execute('''
                CREATE TABLE IF NOT EXISTS user_songs
                (userid int,sessionid int,itemInSession int ,artist text,  song_title text, song_length float,first_name text, last_name text,
                PRIMARY KEY((userid,sessionid),itemInSession))
            ''')
       

In [35]:
'''
Insert data in to the table from csv file
'''
file = 'event_datafile_new.csv'


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # INSERT statements into the `query` variable
        query = "INSERT INTO user_songs (userid,sessionid,itemInSession,artist,song_title,song_length,first_name,last_name)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]
        artist,first_name,gender,itemInSession,last_name,length,level,location,sessionId,song,userId=line
        session.execute(query, (int(userId),artist,int(sessionId),int(itemInSession),song,float(length),first_name,last_name))
        


In [36]:
'''
Select name of artist, song (sorted by itemInSession) and user (first and last name)for userid = 10, sessionid = 182
'''  

rows = session.execute("""SELECT itemInSession, artist, song_title, first_name, last_name FROM user_songs WHERE userid = 10 and sessionid = 182""")

for row in rows:
    print(row.iteminsession, row.artist, row.song_title, row.first_name, row.last_name)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [42]:

'''
Ceating the table according to the query given
Get the user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Columns of the table - song_title , userid,first_name,last_name
PRIMARY KEY consist of partition key song_title and clustering key userid.

'''
session.execute('''
                CREATE TABLE IF NOT EXISTS music_log
                (song_title text, userid int, first_name text, last_name text,
                PRIMARY KEY(song_title,userid))
            ''')
       

In [44]:
'''
Insert the data in to the table from the csv file
'''
file = 'event_datafile_new.csv'


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_log (song_title,userid,first_name,last_name)"
        query = query + "VALUES (%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]
        artist,first_name,gender,itemInSession,last_name,length,level,location,sessionId,song,userId=line
        session.execute(query, (song,int(userId),first_name,last_name))
      


In [45]:
'''
Select every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
'''  

rows = session.execute("""SELECT  first_name, last_name FROM music_log WHERE song_title = 'All Hands Against His Own' """)

for row in rows:
     print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [40]:
## Drop the table before closing out the sessions

session.execute("""DROP TABLE music_log""")

session.execute("""DROP TABLE user_songs""")

session.execute("""DROP TABLE session_songs""")

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()